#### PINN training and validation file for custom loss function weightage (penalty) analysis

In [4]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
# from tensorflow import keras
import tensorflow as tf

# %load_ext tensorboard
# import tensorboard
import os

In [5]:
from datetime import datetime
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

#### Data Paths

In [10]:
BASE_DIR_PATH = '/Users/ge72vep/Desktop/thesis/'
DATA_PATH = 'Data/Model_8/'
MODEL_PATH = 'models/model_8_w_1sec_batch1.h5' ######## Check the data path ######## 
IMAGES_PATH = 'results/images/'
VIDEOS_PATH = 'results/videos/'
EXP_NAME = 'physics_exp_1sec_M8_w_batch1' ######## Check file name ######## 
VIDEO_NAME = 'phy_1sec_M8_w_batch1'  ######## Check file name ######## 

In [11]:
PATH_TO_DATA = os.path.join(BASE_DIR_PATH, DATA_PATH)
save_model_path = os.path.join(BASE_DIR_PATH, MODEL_PATH)
save_images_path = os.path.join(BASE_DIR_PATH, IMAGES_PATH, EXP_NAME)
save_video_path = os.path.join(BASE_DIR_PATH, VIDEOS_PATH, VIDEO_NAME)
save_results_path = os.path.join(BASE_DIR_PATH, 'models', EXP_NAME+'.csv')


#### Loading dataset



In [12]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'))
val_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'val.csv'))

In [14]:
train_df = train_df.sort_values(['identifier','x','time']).reset_index(drop=True)
val_df = val_df.sort_values(['identifier','x','time']).reset_index(drop=True)

In [15]:
X_train = np.array(train_df[['x','time', 'q', 'friction_coeff', 'slope']].values.tolist())
X_val = np.array(val_df[['x','time', 'q', 'friction_coeff', 'slope']].values.tolist())

In [16]:
Y_train = np.array(train_df[['u','h']].values.tolist())
Y_val = np.array(val_df[['u','h']].values.tolist())

#### Params

In [17]:
epochs= 15

#### Model  with grid search

In [18]:
best_val_loss = np.inf
best_model = -1 

In [19]:
results = pd.DataFrame(columns=['n1','n2','n3', 'epochs', 'reg',
                               'val_r2', 'val_nse', 'val_mse', 'val_mae', 'val_mape'])
layer_1_neurons = np.arange(5,26,10) ######## Check the number of neurons ########
layer_2_neurons = np.arange(5,26,10) ######## Check the number of neurons ########
layer_3_neurons = np.arange(5,26,10) ######## Check the number of neurons ########
reg_consts = [0]

#### Weightage to evaluate

In [ ]:
weightage = [1,2,3,4]  ######## Check the weight penalties ########

#### Evaluation Metrics

In [20]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [21]:
def NSE(y_true, y_pred):

    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

#### Custom loss function with weight penalty 

In [22]:
def custom_loss(grads_inputs, weight):
    du_dx, du_dt, dh_dx, fric_coeff, slope = grads_inputs[:,0], grads_inputs[:,1], grads_inputs[:,2], grads_inputs[:,3], grads_inputs[:,4]
    g = K.constant(9.8)
    w =  K.constant(weight)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        loss_saint_venant = du_dt + y_pred[:,0] * du_dx + g*dh_dx + g*slope + g*K.square(fric_coeff) * K.square(y_true[:,0])/(K.pow(y_true[:,1], 4/3) + K.epsilon())
        l = K.mean(K.square(loss_saint_venant))
#         
        return w*l+ K.sum(K.mean(K.square(y_pred - y_true), axis=0))
   
    # Return a function
    return loss

#### Defining Model

In [23]:
for reg in reg_consts:
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                for weight in weightage:
                    print(n1,n2,n3,reg)
                    K.clear_session()
                    model = Sequential()
                    #model.add(Dense(n1, activation = 'relu', input_shape = (1,)))
                    model.add(Dense(n1, activation = 'relu', kernel_regularizer=l2(reg),input_shape = (5,)))
                    model.add(Dense(n2, activation = 'relu', kernel_regularizer=l2(reg)))
                    model.add(Dense(n3, activation = 'relu', kernel_regularizer=l2(reg)))

                    model.add(Dense(2))
                    grads_u = K.gradients(model.output[:,0], model.input)[0]
                    grads_h = K.gradients(model.output[:,1], model.input)[0]


                    du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
                    calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, model.input[:,3],model.input[:,4]), axis=1)
                    # model.summary()
                    #Compile the model
                    model.compile(optimizer = 'adam', loss = [custom_loss(calc_grads_inputs, weight)], metrics=['mape', 'mae', 'mse',NSE, r_square])
                    #fit the model
                    early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
                    history = model.fit(X_train,Y_train, epochs=epochs, batch_size=128, validation_data=(X_val,Y_val), callbacks=[early_stopping_monitor])

                    # Saving results
                    val_loss = history.history['val_loss'][-1]
                    val_mae = history.history['val_mae'][-1]
                    val_mse = history.history['val_mse'][-1]
                    val_mape = history.history['val_mape'][-1]
                    val_nse = history.history['val_NSE'][-1]
                    val_r_square = history.history['val_r_square'][-1]

                    results = results.append({'n1':n1,'n2':n2,'n3':n3, 'epochs':len(history.history['val_loss']),
                                  'reg':reg,'val_r2':val_r_square, 'val_nse':val_nse, 'val_mse':val_mse, 'val_loss':val_loss,
                                            'val_mae':val_mae, 'val_mape':val_mape, 'weight':weight }, ignore_index=True)
                    
                    # For weighted PINN, val_mse can also be used to select the best model
                    if val_loss < best_val_loss: 
                        best_val_loss = val_loss
                        best_model = model
                        best_n1 = n1
                        best_n2 = n2
                        best_n3 = n3
                        best_reg = reg
                        best_history = history
                        model.save(save_model_path)
                        results.to_csv(save_results_path)

results.to_csv(save_results_path)

5 5 5 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 12002500 samples, validate on 2885500 samples
Epoch 1/15
12002500/12002500 [==============================] - 330s 28us/sample - loss: 16.3147 - mape: 50.5862 - mae: 0.5722 - mse: 8.1541 - NSE: -5.7128 - r_square: 0.6599 - val_loss: 0.5252 - val_mape: 29.8980 - val_mae: 0.3882 - val_mse: 0.2584 - val_NSE: 0.7459 - val_r_square: 0.7746mae: 0.6605 - mse: 12.1266 - NSE: -8.9842 - r_ - ETA: 1:28 - loss: 23.4036 - mape: 58.9790 - mae: 0.6519 - mse: 11.6983 - N - ETA: - ETA: 1:21 - loss: 22.5437 - mape: 58.04 - ETA: 1:17 - loss: 22.1546 - map - ETA: 1:12 - loss: 21.6991 - ETA: 42s - loss: 19.0366 - mape: 53.8830 - mae: 0.6042 - mse: 9.5 - ETA: 4s - loss: 16.5913 - mape: 50.9222 - mae: 
Epoch 2/15
12002500/12002500 [==============================] - 324s 27us/sample - loss: 0.3436 - mape: 25.8977 - mae: 0.3153 - mse: 0.1687 - NSE: 0.8620 - r_square: 0.8658 - val_loss: 0.2909 - val_mape: 23.0242 -

12002500/12002500 [==============================] - 452s 38us/sample - loss: 1.4252 - mape: 31.8074 - mae: 0.3857 - mse: 0.7085 - NSE: 0.4187 - r_square: 0.8172 - val_loss: 0.6837 - val_mape: 30.9897 - val_mae: 0.4339 - val_mse: 0.3377 - val_NSE: 0.6672 - val_r_square: 0.6866
Epoch 2/15
12002500/12002500 [==============================] - 451s 38us/sample - loss: 0.1975 - mape: 18.1754 - mae: 0.2341 - mse: 0.0957 - NSE: 0.9217 - r_square: 0.9245 - val_loss: 0.6502 - val_mape: 33.2051 - val_mae: 0.4416 - val_mse: 0.3209 - val_NSE: 0.6838 - val_r_square: 0.6916
Epoch 3/15
12002500/12002500 [==============================] - 458s 38us/sample - loss: 0.1150 - mape: 13.5103 - mae: 0.1709 - mse: 0.0544 - NSE: 0.9554 - r_square: 0.9586 - val_loss: 0.2128 - val_mape: 18.2431 - val_mae: 0.2445 - val_mse: 0.1022 - val_NSE: 0.8993 - val_r_square: 0.9168
Epoch 4/15
12002500/12002500 [==============================] - 450s 37us/sample - loss: 0.0480 - mape: 8.1945 - mae: 0.1028 - mse: 0.0209 - NSE

12002500/12002500 [==============================] - 452s 38us/sample - loss: 0.0254 - mape: 5.5249 - mae: 0.0662 - mse: 0.0096 - NSE: 0.9920 - r_square: 0.9933 - val_loss: 0.1856 - val_mape: 14.3409 - val_mae: 0.2128 - val_mse: 0.0887 - val_NSE: 0.9126 - val_r_square: 0.92626 - mape: 5.7014 - mae: 0.0690 - mse: 0.0102 - NSE:  - ETA: 5:12 - loss: 0.0265 - mape: 5.6929 - mae: 0.0689 - mse: 0.0102 - NSE: 0.9917 - r_square: 0. - ETA: 5:11 - loss: 0.0265 - mape: 5.6901 - mae: 0.0688 - mse: 0.0102 - NSE: 0.9917 - r_square: 0.993 - ETA: 5:11 - loss: 0.0265 - mape: 5.6903 - mae: 0.0688 - mse: 0.0101 - NSE: 0.9917 - r_square: 0.99 - ETA: 5:10 - loss: 0.0265 - mape: 5.6893 - mae: 0.0688 - mse: 0.0101 - NSE: 0.9917 - r_square: 0.9 - ETA: 5:10 - loss: 0.0265 - mape: 5.6879 - mae: 0.0688 - mse: 0.0101 - NSE: 0.9917 - r_square: - ETA: 5:09 - loss: 0.0265 - mape: 5.6833 - mae: 0.0687 - mse: 0.0101 - NSE: 0.9917 - r_square: 0.993 - ETA: 5:09 - loss: 0.0264 - mape: 5.6827 - mae: 0. - ETA: 2:10 - loss:

KeyboardInterrupt: 

#### Model Summary

In [21]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                90        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 12        
Total params: 212
Trainable params: 212
Non-trainable params: 0
_________________________________________________________________
